In [12]:
"""
Détecteur automatique d'orientation pour vidéos
Teste 0°, 90°, 180°, 270° sur plusieurs frames et sauvegarde les résultats
"""

import cv2
import json
from pathlib import Path
import numpy as np

class OrientationManager:
    """Détecte et sauvegarde l'orientation optimale par vidéo"""

    def __init__(self, save_file="camera_orientations.json", sample_frames=50, conf=0.4):
        """
        sample_frames : nombre de frames à tester par rotation
        conf : confiance pour le modèle YOLO
        """
        self.save_file = Path(save_file)
        self.sample_frames = sample_frames
        self.conf = conf
        self.data = self._load()

    def _load(self):
        if self.save_file.exists():
            with open(self.save_file, "r", encoding="utf-8") as f:
                return json.load(f)
        return {}

    def save(self):
        with open(self.save_file, "w", encoding="utf-8") as f:
            json.dump(self.data, f, indent=2)

    def get(self, video_id):
        return self.data.get(video_id)

    def set(self, video_id, info):
        self.data[video_id] = info
        self.save()

    @staticmethod
    def rotate_frame(frame, k):
        if k == 1:
            return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        if k == 2:
            return cv2.rotate(frame, cv2.ROTATE_180)
        if k == 3:
            return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        return frame

    def detect_orientation(self, video_path, model=None, force=True):
        """Teste 0°, 90°, 180°, 270° et retourne la meilleure orientation"""
        video_id = Path(video_path).stem

        # Forcer la détection même si déjà en cache
        if not force:
            cached = self.get(video_id)
            if cached:
                print(f"[ORIENTATION] Chargée depuis cache: {cached['degrees']}°")
                return cached

        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = max(1, total_frames // self.sample_frames)

        results = []

        for k in range(4):  # 0°, 90°, 180°, 270°
            detections_total = 0
            frames_ok = 0

            for i in range(self.sample_frames):
                frame_idx = i * step
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                if not ret:
                    continue

                frame = self.rotate_frame(frame, k)
                detections_total += self._count_persons(frame, model)
                frames_ok += 1

            avg_per_frame = detections_total / max(1, frames_ok)
            results.append({
                "rotation": k,
                "degrees": k*90,
                "detections": detections_total,
                "avg_per_frame": avg_per_frame,
                "frames_tested": frames_ok
            })
            print(f"[ORIENTATION] {k*90:3d}° → {detections_total} personnes sur {frames_ok} frames (avg={avg_per_frame:.2f})")

        best = max(results, key=lambda x: x["detections"])
        cap.release()

        print(f"[ORIENTATION] ✓ Meilleure orientation: {best['degrees']}° ({best['detections']} détections totales)")
        self.set(video_id, best)
        return best

    def _count_persons(self, frame, model=None):
        """Détection rapide: compte les personnes si modèle fourni"""
        if model is None:
            # Fallback: score approximatif selon la taille de l'image
            h, w = frame.shape[:2]
            return (h * w) // 1000000  # score fictif pour tester
        try:
            results = model(frame, conf=self.conf, verbose=False)[0]
            if results.boxes is None:
                return 0
            return sum(int(box.cls[0]) == 0 for box in results.boxes)
        except Exception:
            return 0

# -----------------------------
# Main
# -----------------------------
if __name__=="__main__":
    video_dir = Path("../../data/videos")  # Chemin vers tes vidéos
    videos = sorted(video_dir.glob("*.mp4"))

    orientation_mgr = OrientationManager(sample_frames=50)  # tester 50 frames par vidéo
    
    # Remplace `model=None` par ton modèle YOLO si tu veux de vraies détections
    model = None

    for video in videos:
        print(f"\n▶ Analyse vidéo : {video.name}")
        # Force=True pour toujours refaire la détection
        orientation_mgr.detect_orientation(str(video), model=model, force=True)

    print("\n💾 Résultats sauvegardés dans:", orientation_mgr.save_file)



▶ Analyse vidéo : CAMERA_DEBUT_COULOIR_DROIT.mp4
[ORIENTATION]   0° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION]  90° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 180° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 270° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] ✓ Meilleure orientation: 0° (0 détections totales)

▶ Analyse vidéo : CAMERA_DEVANTURE_PORTE_ENTREE.mp4
[ORIENTATION]   0° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION]  90° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 180° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 270° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] ✓ Meilleure orientation: 0° (0 détections totales)

▶ Analyse vidéo : CAMERA_DEVANTURE_SOUS_ARBRE.mp4
[ORIENTATION]   0° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION]  90° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 180° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 270° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] ✓ Meil

In [13]:
"""
Détecteur automatique d'orientation pour vidéos
Teste 0°, 90°, 180°, 270° sur plusieurs frames, sauvegarde les résultats,
et lit la vidéo avec la rotation détectée pour vérification visuelle.
"""

import cv2
import json
from pathlib import Path
import numpy as np
import time

class OrientationManager:
    """Détecte et sauvegarde l'orientation optimale par vidéo"""

    def __init__(self, save_file="camera_orientations.json", sample_frames=50, conf=0.4):
        self.save_file = Path(save_file)
        self.sample_frames = sample_frames
        self.conf = conf
        self.data = self._load()

    def _load(self):
        if self.save_file.exists():
            with open(self.save_file, "r", encoding="utf-8") as f:
                return json.load(f)
        return {}

    def save(self):
        with open(self.save_file, "w", encoding="utf-8") as f:
            json.dump(self.data, f, indent=2)

    def get(self, video_id):
        return self.data.get(video_id)

    def set(self, video_id, info):
        self.data[video_id] = info
        self.save()

    @staticmethod
    def rotate_frame(frame, k):
        if k == 1:
            return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        if k == 2:
            return cv2.rotate(frame, cv2.ROTATE_180)
        if k == 3:
            return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        return frame

    def detect_orientation(self, video_path, model=None, force=True):
        """Teste 0°, 90°, 180°, 270° et retourne la meilleure orientation"""
        video_id = Path(video_path).stem

        if not force:
            cached = self.get(video_id)
            if cached:
                print(f"[ORIENTATION] Chargée depuis cache: {cached['degrees']}°")
                return cached

        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = max(1, total_frames // self.sample_frames)

        results = []

        for k in range(4):
            detections_total = 0
            frames_ok = 0

            for i in range(self.sample_frames):
                frame_idx = i * step
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                if not ret:
                    continue

                frame = self.rotate_frame(frame, k)
                detections_total += self._count_persons(frame, model)
                frames_ok += 1

            avg_per_frame = detections_total / max(1, frames_ok)
            results.append({
                "rotation": k,
                "degrees": k*90,
                "detections": detections_total,
                "avg_per_frame": avg_per_frame,
                "frames_tested": frames_ok
            })
            print(f"[ORIENTATION] {k*90:3d}° → {detections_total} personnes sur {frames_ok} frames (avg={avg_per_frame:.2f})")

        best = max(results, key=lambda x: x["detections"])
        cap.release()

        print(f"[ORIENTATION] ✓ Meilleure orientation: {best['degrees']}° ({best['detections']} détections totales)")
        self.set(video_id, best)
        return best

    def _count_persons(self, frame, model=None):
        """Détection rapide: compte les personnes si modèle fourni"""
        if model is None:
            # Fallback: score approximatif selon la taille de l'image
            h, w = frame.shape[:2]
            return (h * w) // 1000000
        try:
            results = model(frame, conf=self.conf, verbose=False)[0]
            if results.boxes is None:
                return 0
            return sum(int(box.cls[0]) == 0 for box in results.boxes)
        except Exception:
            return 0

def play_video_with_orientation(video_path, rotation_k):
    """Lit la vidéo avec la rotation correcte pour contrôle visuel"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Erreur: impossible d'ouvrir {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    delay = int(1000 / fps)

    print(f"Lecture de la vidéo avec rotation {rotation_k*90}°... (ESC pour quitter)")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = OrientationManager.rotate_frame(frame, rotation_k)
        cv2.imshow("Video Rotation Control", frame)
        key = cv2.waitKey(delay) & 0xFF
        if key == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    video_dir = Path("../../data/videos")  # Chemin vers tes vidéos
    videos = sorted(video_dir.glob("*.mp4"))

    orientation_mgr = OrientationManager(sample_frames=50)
    model = None  # Remplace par ton modèle YOLO si disponible

    for video in videos:
        print(f"\n▶ Analyse vidéo : {video.name}")
        best_orientation = orientation_mgr.detect_orientation(str(video), model=model, force=True)
        # Lecture pour contrôle visuel
        play_video_with_orientation(str(video), best_orientation["rotation"])

    print("\n💾 Résultats sauvegardés dans:", orientation_mgr.save_file)



▶ Analyse vidéo : CAMERA_DEBUT_COULOIR_DROIT.mp4
[ORIENTATION]   0° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION]  90° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 180° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] 270° → 0 personnes sur 50 frames (avg=0.00)
[ORIENTATION] ✓ Meilleure orientation: 0° (0 détections totales)
Lecture de la vidéo avec rotation 0°... (ESC pour quitter)

▶ Analyse vidéo : CAMERA_DEVANTURE_PORTE_ENTREE.mp4
[ORIENTATION]   0° → 0 personnes sur 50 frames (avg=0.00)


KeyboardInterrupt: 